In [ ]:
import numpy as np

In [265]:
import os
import json
import sys
from argparse import ArgumentParser
import re
import math

In [3]:
file = "test_ft_VxLAN_TrafficBasic.py"
file

'test_ft_VxLAN_TrafficBasic.py'

In [4]:
with open(file, 'r') as file_pointer:
    test_contents = file_pointer.read()

In [5]:
test_contents

'# (c) Copyright 2019 Hewlett Packard Enterprise Development LP\n# All Rights Reserved.\n#\n# The contents of this software are proprietary and confidential\n# to the Hewlett Packard Enterprise Development LP. No part of this\n# program may be photocopied, reproduced, or translated into another\n# programming language without prior written consent of the\n# Hewlett Packard Enterprise Development LP.\n\nfrom retrying import retry\nfrom topology_common.utils.threading import ThreadGroup\nfrom time import sleep\nfrom pytest import (\n    mark,\n    fixture\n)\n\nfrom topology_common.ops.vxlan.ft_vxlan_lib import (\n    PortType,\n    TrafficType,\n    verify_mac_table\n)\nfrom topology_common.ops.l3_common.ft_l3_lib import (\n    AddressType\n)\nfrom topology_common.ops.vlans.vlan import configure_switch_vlan\n\nfrom fixtures import (\n    env_setup_hs_sw_hs,\n    l2_basic_phy,\n    l3_basic_phy)\n\n__doc__ = """\nTest Case Header:\n=================\n:Author: Surbhit Sinha - surbhit.sinh

In [ ]:
output = parse_test_file(test_contents, os.path.dirname(test_script))

In [7]:
topology_re = r'Topology:\n=+\n\.\.\sditaa::\n(?P<Topology>.*)$'

In [18]:
topology_re = r'(?P<TOPO>TOPOLOGY\s=\s\"\"\"\n.*?target=\"true\".*?\n\s*\"\"\"\n?)'

In [24]:
re_result = re.search(topology_re, test_contents, flags=(DOTALL | IGNORECASE)).group(0)
re_result

'TOPOLOGY = """\n# Nodes\n[type=halon_0 name="OpenSwitch 1" target="true"] ops1\n[type=host name="Host 1" image="ubuntuscapy_2.4:latest"] hs1\n[type=host name="Host 1" image="ubuntuscapy_2.4:latest"] hs2\n\n# Links\nhs1:eth1 -- ops1:if01\nops1:if02 -- hs2:eth1\n"""\n'

# Creating Workspace

In [292]:
grid = []
for i in range(20):
    temp =[]
    for j in range(20):
        temp.append(' ')
    grid.append(temp)
        
grid

[[' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' '],
 [' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' '],
 [' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' '],
 [' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' '],
 [' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' '],
 [' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' '],
 [' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' '],
 [' ',
  ' ',

In [293]:
def draw_switch(grid, dev_name, coord, sw_width, sw_height):
    row, col = coord
    
    r_cent = row + (sw_height//2)
    c_cent = col + (sw_width//2)
    
    grid[r_cent][c_cent] = dev_name
    
    grid[row][col] = '+'
    grid[row+sw_height][col] = '+'
    grid[row][col+sw_width] = '+'
    grid[row+sw_height][col+sw_width] = '+'
    # Draw top horizontal line
    for i in range(col+1, col + sw_width):
        grid[row][i] = '-'
    # Draw bottom horizontal line
    for i in range(col+1, col + sw_width):
        grid[row+sw_height][i] = '-'
    
    # Draw left vertical line
    for i in range(row+1, row + sw_height):
        grid[i][col] = '|'
    
    # Draw right vertical line
    for i in range(row+1, row + sw_height):
        grid[i][col+sw_width] = '|'
        
    xtra = len(dev_name)
    #diff = sw_width - xtra
    sub = math.ceil(xtra/2)
    
    print("center= ", r_cent, c_cent)
    print(sub)
    
    temp = grid[r_cent]
    for i, j in zip(range(c_cent-sub, c_cent), range(c_cent+1, c_cent+sub)):
        del grid[r_cent][i]    
        del grid[r_cent][j]
    
    return grid
    
    

In [296]:
sw_width = 10
sw_height = 5

coord = (2, 2)


hs_width = 1
hs_height = 1

hs_coord = (0, 0)
new_grid = draw_switch(grid, "switch", coord, sw_width, sw_height)
new_grid2 = draw_switch(new_grid, "hs1", hs_coord, hs_width, hs_height)
new_grid2

center=  4 7
3
center=  0 0
2


[['+',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' '],
 ['+',
  '+',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' '],
 [' ',
  ' ',
  '+',
  '-',
  '-',
  '-',
  '-',
  '-',
  '-',
  '-',
  '-',
  '-',
  '+',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' '],
 [' ',
  ' ',
  '|',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  '|',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' '],
 [' ', ' ', '|', ' ', ' ', 'switch', ' ', '|', ' ', ' ', ' ', ' ', ' ', ' '],
 [' ',
  ' ',
  '|',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  '|',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' '],
 [' ',
  ' ',
  '|',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  '|',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' '],
 [' ',
  ' ',
  '+',
  '-',
  '-',
  '-',
  '-',
  '-',
  '-',
  '-',
  '-',
  '-',
  '+',
  ' ',
  ' ',

In [297]:
print('\n'.join([''.join(l) for l in new_grid2]))

+               
++                  
  +---------+       
  |         |       
  |  switch |      
  |         |       
  |         |       
  +---------+       
                    
                    
                    
                    
                    
                    
                    
                    
                    
                    
                    
                    


In [284]:
please = ''.join(''.join(l) for l in new_grid2)
please

'+                 ++                    +---------+         |         |         |  switch            |         |         |         |         +---------+                                                                                                                                                                                                                                                       '

In [126]:
def draw_grid(grid, dev_name):
    if dev_name == "switch":
        draw_switch(grid, dev_name, )
    array_len = len(lst)//2 - 1
    array_width = len(lst[0])-2
    sw_width = 5
    sw_height = 4
    start_coord = 3
    # Draw the line
    for i in range(2, sw_width):
        lst[1][i] = "-"
    for j in range(2, sw_height):
        lst[j][3] = "|"
    return lst

In [127]:
dev_name = 'hs1'
new_lst = draw_switch(lst, dev_name)
please_work_please = '\n'.join(''.join(l) for l in new_lst)
print(please_work_please)

..........
..---.....
...|......
..||......
..........
..........
..........
..........
..........
..........


In [93]:
new_lst

[[' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' '],
 [' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' '